In [3]:
from math import sqrt
from statistics import mean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pp
import itertools

/Users/samueleceol/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("CRISP/CSV/db/farming_system.csv")
df

,Farming system,Macro region,Agro-ecological zone,Total area (m ha),Cultivated area (m ha),Cattle population (m head),Irrigated area (m ha),Total population (m),Agricultural population (m),Farm size (ha),Description,Irrigated/rainfed
0,Lowland Rice,EAP,Humid and Moist Sub-humid,197,71,52,33,825,474,197,The lowland rice farming system is found in bo...,Both
1,Temperate Mixed,EAP,Dry Sub-humid,99,,,,,,6.5,Temperate mixed farming system is found in moi...,Both
2,Upland Intensive Mixed,EAP,Humid and Sub-humid,314,,,,,,,The upland intensive mixed agricultural system...,Rainfed
3,Coastal plantation and mixed,LAC,,186,,,,,,,Coastal plantation and mixed farming in the La...,Rainfed
4,Dryland mixed,LAC,Dry-moist Sub-humid,127,,,,,,3.5,The Dryland Mixed Farming System includes two ...,Rainfed
5,Forest based Livestock,LAC,,600,,,,,,,The forest based agricultural system is locate...,Rainfed
6,High Altitude Mixed (C. Andes),LAC,Semi-arid Dry Sub-humid,121,,,,,,3,The maize-beans agricultural system covers pre...,Rainfed
7,Irrigated,LAC,Arid and Semi-arid,200,,,,,,,The irrigated agricultural system covers large...,Irrigated
8,Maize-beans (Mesoamerica),LAC,Dry-moist Sub-humid,65,,,,,,2.55,The maize-beans agricultural system covers pre...,Rainfed
9,Dryland mixed,MENA,Semi-arid Sub-humid,42,3.84,2.59,,,,7,The dryland mixed agricultural system of the M...,Rainfed


In [5]:
def PLI_single(df, col_idx):
    res = {}
    for val in df.iloc[:, col_idx].unique():
        pli = (df[df.iloc[:, col_idx] == val].index.tolist())
        if len(pli) > 1:
            res[list(df.iloc[:, col_idx].unique()).index(val)] = pli
    return res

In [6]:
def PLI_intersect(p1, p2):
    hash = {}
    gid = 1

    for group in p1:
        for row in p1[group]:
            hash[row] = gid
        gid += 1
    
    pli = {}
    gid = 1
    for group in p2:
        for row in p2[group]:
            if(row in hash):
                key = [hash[row], gid]

                if(tuple(key) not in pli):
                    pli[tuple(key)] = [row]
                else:
                    pli[tuple(key)] = pli[tuple(key)] + [row]
        gid += 1
    
    return {k: v for k, v in pli.items() if len(v) > 1}

PLI_intersect({1:[1,2], 2:[0,4]}, {1:[1,2,3,4], 4:[1,4]})

{(1, 1): [1, 2]}

In [122]:
#15_11
def UCC_discovery(df):
    k = 0
    E = []
    F = []
    pli_list = []
    for col_idx in range(0, len(df.columns)): 
        cur_pli = PLI_single(df, col_idx)
        if cur_pli: pli_list.append([[col_idx], cur_pli]) # Add only non-empty returns
    E.insert(k, pli_list)
    F.insert(k, UCC_prune(E[k]))

    while len(F[k]) != 0:
        E.insert(k+1, UCC_candidates(F[k]))
        F.insert(k+1, UCC_prune(E[k+1]))
        k += 1

    #-------
    result = []
    for j in range(k):
        result += [x for x in E[j] if x not in F[j]]
    return result
    #-------

def UCC_prune(E):
    F = []
    for e in E:
        if e[1]: F.append([e[0], e[1]])
    return F

def UCC_candidates(F):
    E = []
    #print(F)
    for f_1, p_1 in F:
        for f_2, p_2 in F:
            # Column combination differs only by one and
            # Last column idx for f_1 < Last column idx for f_2
            if((f_1[:-1] == f_2[:-1]) and (f_1[-1] < f_2[-1])):
                f = f_1 + f_2[-1:]
                
                # Check subsets of f of size len(f) - 1
                if(all(item in list(zip(*F))[0] for item in [list(t) for t in itertools.combinations(f, len(f)-1)])):
                    p = PLI_intersect(p_1, p_2)
                    E.append([f, p])
    return E


In [123]:
UCC_discovery(df)

[[[0, 1], {}],
 [[0, 2], {}],
 [[0, 9], {}],
 [[0, 10], {}],
 [[2, 10], {}],
 [[9, 10], {}]]

In [135]:
#13_12
def FD_discovery():


IndentationError: expected an indented block (<ipython-input-135-c2fcc78dbaba>, line 2)